In [ ]:
import os

## Step 1: Importing the Relevent Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### Generic functions to be used during Exploratory Data Analysis

## Step 2: Data Inspection and Cleaning

In [ ]:
df_train_org = pd.read_csv('/kaggle/input/jobathon-nov2021/train_MpHjUjU.csv')
df_test_org = pd.read_csv('/kaggle/input/jobathon-nov2021/test_hXY9mYw.csv')

In [ ]:
df_train_org.shape, df_test_org.shape

In [ ]:
df_train_org.head()

In [ ]:
df_test_org.head()

**Except Emp_ID column, No other column is available for test dataset. Have to extract test dataset feature from train dataset using JOIN by Emp_ID.**

In [ ]:
df_train_org.info()

In [ ]:
df_train_org.isnull().sum() / df_train_org.shape[0] * 100

**We have more than 91% of missing values for column LastWorkingDate. This is because of two reasons:**

In [ ]:
df_train_org['Emp_ID'].nunique()  

In [ ]:
df_train_org['Emp_ID'].nunique()   -  df_test_org['Emp_ID'].nunique()

In [ ]:
df_train_org['MMM-YY'] = pd.to_datetime(df_train_org['MMM-YY'])
df_train_org['Dateofjoining'] = pd.to_datetime(df_train_org['Dateofjoining'])
df_train_org['LastWorkingDate'] = pd.to_datetime(df_train_org['LastWorkingDate'])
df_train_org.head() 

In [ ]:
df_train_org.info()

## Solution Approch

Using Supervised Learning, train a Machine Learning model using half-yearly (6 month) sales and performance data.
Calculate the probability of an Employee resigning in the next 6 month

## Step 3: Feature Engineering

**Divided given 2 years of data points in 4 half-yearly dataset as “2016_HY1, 2016_HY2, 2017_HY1, 2017_HY2” respectively. **

In [ ]:
df_train_11 = df_train_org.loc[df_train_org['MMM-YY'] < '2016-7-1']
df_train_11.shape

In [ ]:
df_train_11.head()

In [ ]:
df_train_12 = df_train_org.loc[(df_train_org['MMM-YY'] > '2016-6-30') & (df_train_org['MMM-YY'] < '2017-1-1')]
df_train_12.shape

In [ ]:
df_train_21 = df_train_org.loc[(df_train_org['MMM-YY'] > '2016-12-31') & (df_train_org['MMM-YY'] < '2017-6-30')]
df_train_21.shape

In [ ]:
df_train_22 = df_train_org.loc[(df_train_org['MMM-YY'] > '2017-6-30') & (df_train_org['MMM-YY'] < '2018-1-1')]
df_train_22.shape

- Created new features like total experience, number of half-yearly promotions, minimum and maximum Quarterly rating and a dichotomous feature Resigned.
- Updated existing features like Average half-yearly sales value.
- Encoded existing feature like Education, Gender.

- Train Dataset

In [ ]:
#olumns = ['emp_id', 'age', 'gender', 'city', 'education', 'salary', 'j_designation', 'c_designation', 'avg_sales_value', 'min_rating', 'max_rating']
columns = ['emp_id', 'age', 'gender', 'city', 'education', 'salary', 'j_designation', 'avg_sales_value', 'min_rating', 'max_rating']
columns.extend(['np_of_promotion', 'experience', 'Resigned'])
df_train = pd.DataFrame(columns = columns)
df_train.head()

In [ ]:
def generate_dataset(df_h1, df_h2):
  max_date = df_h2['MMM-YY'].min()
  Emp_ID_list = df_h1['Emp_ID'].unique()
  index = df_train.shape[0]
  for emp_id in Emp_ID_list:
    #'''
    if df_train.loc[df_train['emp_id'] == emp_id].shape[0] > 0:
      if df_train.loc[df_train['emp_id'] == emp_id, 'Resigned'].values[-1] == 1:
        continue
    #'''
    df_temp_1 = df_h1.loc[df_h1['Emp_ID'] == emp_id]
    eid = emp_id
    age = df_temp_1.Age.max()
    gender = 1 if df_temp_1.iloc[0,3] == 'Male' else 0
    city = df_temp_1.iloc[0,4]
    eduction_str = df_temp_1.iloc[0,5]
    if eduction_str == 'College':
      education = 0
    elif eduction_str == 'Bachelor':
      education = 1
    else:
      education = 2
    salary = df_temp_1['Salary'].max()
    j_designation = df_temp_1.iloc[0,9]
    c_designation = df_temp_1['Designation'].max()
    avg_sales_value = df_temp_1['Total Business Value'].mean()
    min_rating = df_temp_1['Quarterly Rating'].min()
    max_rating = df_temp_1['Quarterly Rating'].max()
    np_of_promotion = c_designation - j_designation
    df_temp2 = df_h2.loc[df_h2['Emp_ID'] == emp_id]
    if df_temp2.shape[0] == 0:
      lwd = df_temp_1.iloc[-1, 8]
      if pd.isnull(lwd):
        #lwd = max_date
        lwd = df_temp_1.iloc[-1,7]
        #continue
      resigned = 1
      experience = (lwd -  df_temp_1.iloc[0,7]).days
    elif not pd.isnull(df_temp2.iloc[-1, 8]):
      lwd = df_temp2.iloc[-1, 8]
      resigned = 1
      experience = (lwd -  df_temp_1.iloc[0,7]).days
    else:
      resigned = 0
      experience = (max_date -  df_temp_1.iloc[0,7]).days
    #df_train.loc[index] = eid, age, gender, city, education, salary, j_designation, c_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience, resigned
    df_train.loc[index] = eid, age, gender, city, education, salary, j_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience, resigned
    index += 1
    


In [ ]:
generate_dataset(df_train_11, df_train_12)
df_train.shape

In [ ]:
generate_dataset(df_train_12, df_train_21)
df_train.shape

In [ ]:
generate_dataset(df_train_21, df_train_22)
df_train.shape

In [ ]:
def generate_last_dataset(last_dataframe):
  list_1 = last_dataframe['Emp_ID'].unique()
  list_2 = df_test_org['Emp_ID'].unique()
  Emp_ID_list = list(set(list_1) - set(list_2)) + list(set(list_2) - set(list_1))
  index = df_train.shape[0]
  for emp_id in Emp_ID_list:
    #'''
    if df_train.loc[df_train['emp_id'] == emp_id].shape[0] > 0:
      if df_train.loc[df_train['emp_id'] == emp_id, 'Resigned'].values[-1] == 1:
        continue
    #'''
    df_temp_1 = last_dataframe.loc[last_dataframe['Emp_ID'] == emp_id]
    eid = emp_id
    age = df_temp_1.Age.max()
    gender = 1 if df_temp_1.iloc[0,3] == 'Male' else 0
    city = df_temp_1.iloc[0,4]
    eduction_str = df_temp_1.iloc[0,5]
    if eduction_str == 'College':
      education = 0
    elif eduction_str == 'Bachelor':
      education = 1
    else:
      education = 2
    salary = df_temp_1['Salary'].max()
    j_designation = df_temp_1.iloc[0,9]
    c_designation = df_temp_1['Designation'].max()
    avg_sales_value = df_temp_1['Total Business Value'].mean()
    min_rating = df_temp_1['Quarterly Rating'].min()
    max_rating = df_temp_1['Quarterly Rating'].max()
    np_of_promotion = c_designation - j_designation

    lwd = df_temp_1.iloc[-1, 8]
    resigned = 1
    if pd.isnull(lwd):
      lwd = df_temp_1.iloc[-1, 7]
    experience = (lwd -  df_temp_1.iloc[0,7]).days
    #df_train.loc[index] = eid, age, gender, city, education, salary, j_designation, c_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience, resigned
    df_train.loc[index] = eid, age, gender, city, education, salary, j_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience, resigned
    index += 1
    
generate_last_dataset(df_train_22)

In [ ]:
df_train.shape

In [ ]:
duplicate = df_train[df_train.duplicated()]
duplicate.shape

In [ ]:
df_train.drop_duplicates(keep = False, inplace = True)

In [ ]:
df_train.shape

In [ ]:
np.bincount(df_train['Resigned'])

In [ ]:
df_train.sort_values('emp_id').head()

- Test Dataset

In [ ]:
#test_columns = ['emp_id', 'age', 'gender', 'city', 'education', 'salary', 'j_designation', 'c_designation', 'avg_sales_value', 'avg_rating']
test_columns = ['emp_id', 'age', 'gender', 'city', 'education', 'salary', 'j_designation', 'avg_sales_value', 'min_rating', 'max_rating']
test_columns.extend(['np_of_promotion', 'experience'])
df_test = pd.DataFrame(columns = test_columns)

Emp_ID_list = df_test_org['Emp_ID'].unique()
for index, emp_id in enumerate(Emp_ID_list):
  df_temp_1 = df_train_22.loc[df_train_22['Emp_ID'] == emp_id]
  eid = emp_id
  age = df_temp_1.Age.max()
  gender = 1 if df_temp_1.iloc[0,3] == 'Male' else 0
  city = df_temp_1.iloc[0,4]
  eduction_str = df_temp_1.iloc[0,5]
  if eduction_str == 'College':
    education = 0
  elif eduction_str == 'Bachelor':
    education = 1
  else:
    education = 2
  salary = df_temp_1['Salary'].max()
  j_designation = df_temp_1.iloc[0,9]
  #j_designation = df_temp_1['Designation'].min()
  c_designation = df_temp_1['Designation'].max()
  avg_sales_value = df_temp_1['Total Business Value'].mean()
  min_rating = df_temp_1['Quarterly Rating'].min()
  max_rating = df_temp_1['Quarterly Rating'].max()
  np_of_promotion = c_designation - j_designation
  experience = (datetime.datetime(2018,1,1) -  df_temp_1.iloc[0,7]).days
  #df_test.loc[index] = eid, age, gender, city, education, salary, j_designation, c_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience
  df_test.loc[index] = eid, age, gender, city, education, salary, j_designation, avg_sales_value, min_rating, max_rating, np_of_promotion, experience

In [ ]:
df_test.head()

## Step 4: Exploratory Data Analysis

In [ ]:
result = list()
list_1 = df_train_org['Emp_ID'].unique()
list_2 = df_test_org['Emp_ID'].unique()
for eid in list_1:
  df_temp = df_train_org.loc[df_train_org['Emp_ID'] == eid]
  lwd = df_temp.iloc[-1, 8]
  if pd.isnull(lwd):
    if eid not in list_2:
      result.append(eid)
print("Absconding", result)

In [ ]:
df_corr = df_train.copy()
for col in df_corr.columns:
  if col not in ['city', 'gender', 'emp_id']:
    df_corr[col] = df_corr[col].astype(int)
df_corr.corr()

## Step 4: Feature Scaling

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [ ]:
#numeric_col = ['age', 'education', 'salary', 'j_designation', 'c_designation', 'avg_sales_value', 'min_rating', 'max_rating', 'np_of_promotion', 'experience']
numeric_col = ['age', 'education', 'salary', 'j_designation', 'avg_sales_value', 'min_rating', 'max_rating', 'np_of_promotion', 'experience']

In [ ]:
df_train_numeric = df_train[numeric_col]
df_test_numeric = df_test[numeric_col]

In [ ]:
scaler = StandardScaler().fit(df_train_numeric)
X = pd.DataFrame(scaler.transform(df_train_numeric), columns = numeric_col)
X_test = pd.DataFrame(scaler.transform(df_test_numeric), columns = numeric_col)


In [ ]:
X.shape, X_test.shape

In [ ]:
X['gender'] = df_train['gender']
X_test['gender'] = df_test['gender']
X['city'] = df_train['city']
X_test['city'] = df_test['city']
y = df_train['Resigned']

In [ ]:
X.shape, X_test.shape

In [ ]:
X = pd.get_dummies(X, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

In [ ]:
y = np.asarray(y).astype('float32')

In [ ]:
X.shape, X_test.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=123)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

## Step 5: Model building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers, regularizers

In [ ]:
model = Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=[X.shape[1]]))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer = optimizers.Adam(), loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
#history = model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = 32, epochs = 100, callbacks=EarlyStopping(patience=5), verbose = 1)
history = model.fit(X, y, batch_size = 32, epochs = 19, verbose = 1)

## Step 6: Submission

In [ ]:
df_submission = pd.read_csv('/kaggle/input/jobathon-nov2021/sample_submission_znWiLZ4.csv')
df_submission.head()

In [ ]:
df_submission['Target'] = list(model.predict(X_test)[:,0])
df_submission['Target'] = df_submission['Target'].apply(lambda x: 0 if x<0.5 else 1)
df_submission.to_csv('my_submission_NN_ELU.csv', index = False)
np.bincount(df_submission.Target)

In [ ]:

from keras.models import model_from_yaml

# Serialize classification model to YAML
class_yaml = model.to_json()
with open("M_RELU.yaml", "w") as yaml_file:
    yaml_file.write(class_yaml)
# Serialize weights to HDF5
model.save_weights("M_RELU_weights.h5")
print("Saved classifier model to disk")
